### Import packages

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
rayashop_link = "https://www.rayashop.com/en/computers/laptops/"

# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
    
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
    
    return web


# scroll down all the page to load all data of item                       
def scroll_down_all_the_way():
    
    from time import sleep
    
    global scroll_delay_s
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'): # end if reach to end of page
        start = end
        # step by step
        end = start + 1000
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(2)
        
# get all links of laptops            
def get_laptops_links():
    
    soup = BeautifulSoup(web.page_source, 'html.parser')
    
    parent_tag = soup.find("div", id="amasty-shopby-product-list")
    tag = parent_tag.find_all("div", class_="product-item-info")
    
    links = [x.a["href"] for x in tag]
    
    return links


# scrape data and append it in list of dictionary
def get_laptops_info(link):
    time.sleep(2)
    web.get(link)
    listing = {}
    listing["link"] = link
    soup = BeautifulSoup(web.page_source, 'html.parser')
    

    
    tag = soup.find("div", class_="product-info-main")
    if tag:
        listing["title"] = tag.find("div", class_="page-title-wrapper product").h1.get_text(strip=True)
        listing["price"] = tag.find("div", class_=re.compile("final_price$")).get_text(strip=True)
                
        
    tag = soup.find("div", class_="product-specs")
    feature = tag.find_all("div", class_="feature")
    
    # Concate all features and spliting it by ";" 
    if feature:
        listing["feature"] = " ; ".join([x.text for x in feature])
        
     # to save data in dectionary       
    description = soup.find("div", class_="product attribute description").find("div" ,class_="value").ul.find_all("li")
    if description:
        description = [descripe.get_text(strip=True).split(": ") for descripe in description]
        
        global values
        global keys

        values = []
        keys = []
        for i in description:
            values.append(i[-1])
            keys.append(i[0])
    
        dict_value = dict(zip(keys, values))
        listing.update(dict_value)
        
    info = soup.find("div", class_="gazri-product-view-block gazri-product-attributes").find_all("tr")
    if info:
        more_info = [item.text for item in info]
        listing['more_info'] = ' ; '.join([features for features in more_info])

    print(link)
    print(f"{len(listing)} - ", listing)
    print("-" *100, "\n\n")
    
    return listing


# to save all operation on PC
def save_state():
    
    import joblib
    
    global listings
    global state_filepath
    
    state = listings
    joblib.dump(state, state_filepath)

In [3]:
state_filepath = r"E:\other\Epsilon_Ai\Final Project\final project\rayashop.pkl"
listings = []
web = open_chromedriver()
web.get(rayashop_link)       
scroll_down_all_the_way()
links = get_laptops_links()



while links:# to git all links
    try:
        link = links.pop()
        listing = get_laptops_info(link)
        listings.append(listing)
        
    except:# to make sure all informations is loaded
        link = links.pop()
        listing = get_laptops_info(link)
        listings.append(listing)
 
save_state()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\Mora\AppData\Local\Temp/ipykernel_17212/1857018584.py:15: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


https://www.rayashop.com/en/hp-envy-x360-laptop-intel-core-i7-1195g7-13-3-inch-fhd-touch-512gb-ssd-8gb-ram-windos-11-silver
15 -  {'link': 'https://www.rayashop.com/en/hp-envy-x360-laptop-intel-core-i7-1195g7-13-3-inch-fhd-touch-512gb-ssd-8gb-ram-windos-11-silver', 'title': 'HP ENVY X360 Laptop, Intel Core i7-1195G7, 13.3 Inch FHD Touch, 512GB SSD, 8GB RAM, Windos 11 - Silver', 'price': 'EGP22,900.00', 'feature': 'Graphics Card BrandIntel ; Hard Disk Capacity512 GB ; Operating SystemWindows ; RAM Capacity8 GB ; Screen Size13 - 13.9 Inch ; Processor TypeIntel Core i7-1195G7 ; BrandHP ; Product ColorSilver', 'Processor Family': 'Intel Core i7', 'Processor Type': 'Core i7-1195G7', 'Processor Core': 'Quad-Core', 'Screen Size': '13.3 Inch', 'Hard Disk Capacity': '512 GB', 'RAM Capacity': '8 GB', 'Graphics Card Brand': 'Intel', 'Graphics Card Capacity': 'Intel Iris graphics', 'Operating System': 'Windows 11', 'Color': 'Silver', 'more_info': ' Seller SKU EHPenvBD1033DX-Elite  ;  Brand HP  ;  

In [4]:
print(link)

https://www.rayashop.com/en/dell-g15-5511-gaming-laptop-intel-core-i7-11800h-15-6-inch-fhd-512gb-ssd-16gb-ram-nvidia-geforce-rtx-3050ti-4g-ubuntu-grey


In [9]:
df = pd.DataFrame.from_records(listings)
df

,link,title,price,feature,Processor Family,Processor Type,Processor Core,Screen Size,Hard Disk Capacity,RAM Capacity,...,RAM Type,10th Generation Intel Core i3 Processor,Processor Number,Cache,Model Name,Hard Desk Capacity,Ram Capacity,HDMI Port,Usage,Battery Capacity
0,https://www.rayashop.com/en/hp-envy-x360-lapto...,"HP ENVY X360 Laptop, Intel Core i7-1195G7, 13....","EGP22,900.00",Graphics Card BrandIntel ; Hard Disk Capacity5...,Intel Core i7,Core i7-1195G7,Quad-Core,13.3 Inch,512 GB,8 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.rayashop.com/en/huawei-matebook-d-...,"HUAWEI MateBook D 14 Laptop, AMD Ryzen 5 3500U...","Special PriceEGP17,199.00Regular PriceEGP18,69...",Graphics Card BrandAMD ; Hard Disk Capacity512...,AMD,AMD Ryzen 5 3500U,Quad Core,14 Inch,512GB,8GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.rayashop.com/en/catalog/product/vi...,"Lenovo ThinkPad E14 Notebook, Intel Core i7-10...","Special PriceEGP14,234.42Regular PriceEGP15,14...",Graphics Card BrandAMD ; Hard Disk Capacity1 T...,Intel Core i7,Intel Core i7-10510U,Quad-Core,14 - 14.9 Inch,1 TB,8 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.rayashop.com/en/lenovo-thinkpad-e1...,"Lenovo ThinkPad E14 G2, Intel Core i5 1135G7, ...","EGP17,000.00",Graphics Card BrandNVIDIA ; Hard Disk Capacity...,NaN,Core i5 1135G7,Quad-Core,14 Inch,256GB,8GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.rayashop.com/en/lenovo-v15-ada-lap...,"Lenovo V15-ADA Laptop, AMD 3020e, 15.6 inch HD...","Special PriceEGP5,935.00Regular PriceEGP6,999.00",Graphics Card BrandAMD ; Hard Disk Capacity1 T...,NaN,AMD 3020e,Dual-Core,15.6 Inch,1TB,4GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,https://www.rayashop.com/en/asus-tuf-dash-f15-...,"ASUS TUF Dash F15 FX516PM-HN023W, Intel Core I...","Special PriceEGP27,560.00Regular PriceEGP29,60...",Graphics Card BrandNVIDIA ; Hard Disk Capacity...,NaN,Intel Core I7-11370H,Quad,15.6 Inch,512GB,16GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,https://www.rayashop.com/en/asus-tuf-dash-f15-...,"ASUS TUF Dash F15 FX516PC-HN558T, Intel Core I...","Special PriceEGP18,200.00Regular PriceEGP21,00...",Graphics Card BrandNVIDIA ; Hard Disk Capacity...,NaN,Intel Core I5-11300H,Quad,15.6 Inch,512GB,8GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,https://www.rayashop.com/en/asus-tuf-dash-f15-...,"ASUS TUF Dash F15 FX516PC-HN001T, Intel Core I...","Special PriceEGP23,920.00Regular PriceEGP24,20...",Graphics Card BrandNVIDIA ; Hard Disk Capacity...,NaN,Intel® Core I7-11370H,Quad,15.6 Inch,512GB,8GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,https://www.rayashop.com/en/catalog/product/vi...,"Acer Aspire 3 A315-56-32BJ Laptop, Intel Core ...","Special PriceEGP8,455.00Regular PriceEGP8,735.00",Graphics Card BrandIntel ; Hard Disk Capacity1...,NaN,Intel Core i3-1005G1,Dual,NaN,1TB,4GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
